# Download images

In [ ]:
#Make sure to use following gid instead of main one:
#pip3 install git+https://github.com/Joeclinton1/google-images-download.git
import os
import shutil

## Collect lists of classes

In [4]:
cwd = os.getcwd()
data_dir = cwd + '/new_cars'
limit = 100
train_ratio = 0.8
model = 'Mclaren 650S'

anno_train = []
anno_test = []
names = []
cars_tba = []

for line in open(cwd + '/naming/anno_train.csv'):
    anno_train.append(line)
for line in open(cwd + '/naming/anno_test.csv'):
    anno_test.append(line)
for line in open(cwd + '/naming/new_names.csv'):
    cars_tba.append(line.strip('\n'))

## The functions are defined here

In [15]:
from google_images_download import google_images_download as gid

def download_images(model_list, directory, limit=100):
    for model in model_list:
        if not os.path.exists(f'{directory}/new/{model}') and not os.path.exists(f'{directory}/train/{model}'):
            args = {'keywords':model, 
                    'limit':limit, 
                    'format':'jpg', 
                    'output_directory':directory + '/new', 
                    'image_directory':model,
                    'size':'medium',
                    'suffix_keywords':'exterior'}
            response = gid.googleimagesdownload()
            paths = response.download(args)
            for path in paths[0][model + ' exterior']:
                split = path.split('.')
                if split[len(split) - 1] != 'jpg':
                    os.remove(path)
        
def rename_images(directory, model, data_list):
    directory_model = directory + '/' + model
    for filename in os.listdir(directory_model):
        last_file_number = int(data_list[len(data_list) - 1].split(',')[0].split('.')[0])
        new_name = '%0*d' %(5, last_file_number + 1) + '.jpg'
        os.rename(directory_model + '/' + filename, directory_model + '/' + new_name)
        data_list.append(new_name + ',0')
            
def move_images(directory, model):
    dir_list = os.listdir(f'{directory}/new/{model}')
    print(dir_list)
    if not os.path.exists(f'{directory}/train/{model}'):
        os.mkdir(f'{directory}/train/{model}')
    if not os.path.exists(f'{directory}/test/{model}'):
        os.mkdir(f'{directory}/test/{model}')
    
    for i, filename in enumerate(dir_list):
        if i < int(len(dir_list) * train_ratio):
            shutil.copyfile(directory + '/train_test/' + model + '/' + filename, directory + '/train/' + model + '/' + filename)
        else:
            shutil.copyfile(directory + '/train_test/' + model + '/' + filename, directory + '/test/' + model + '/' + filename)

def redistribute_images(directory, cars_tba, dir1='/train/', dir2='/test/', train_test_ratio=0.8):
    if not os.path.exists(directory + dir1[:len(dir1) - 1]):
        os.mkdir(directory + dir1[:len(dir1) - 1])        
    if not os.path.exists(directory + dir2[:len(dir1) - 1]):
        os.mkdir(directory + dir2[:len(dir1) - 1])
            
    for model in cars_tba:
        if not os.path.exists(directory + dir1 + model):
            os.mkdir(directory + dir1 + model)
        if not os.path.exists(directory + dir2 + model):
            os.mkdir(directory + dir2 + model)
        train_list = os.listdir(directory + dir1 + model)
        test_list = os.listdir(directory + dir2 + model)

        for element in test_list:
            train_list.append(test_list.pop(0))
        
        total = len(train_list)
        while len(test_list) < (total * (1 - train_test_ratio)):
            test_list.append(train_list.pop(0))
        for filename in train_list:
            if os.path.exists(directory + dir2 + model + '/' + filename):
                shutil.move(directory + dir2 + model + '/'  + filename, directory + dir1 + model + '/'  + filename)
        for filename in test_list:
            if os.path.exists(directory + dir1 + model + '/' + filename):
                shutil.move(directory + dir1 + model + '/'  + filename, directory + dir2 + model + '/'  + filename)
            
def remove_nonjpeg(directory, model):
    train_list = os.listdir(directory + '/train/' + model)
    test_list = os.listdir(directory + '/test/' + model)
    for filename in train_list:
        if '.jpg' not in filename:
            os.remove(directory + '/train/' + model + '/' + filename)
            removed += 1
    for filename in test_list:
        if '.jpg' not in filename:
            os.remove(directory + '/test/' + model + '/' + filename)
            removed += 1

## Download new images

In [12]:
for model in cars_tba:
    if not os.path.exists(data_dir + '/train_test/' + model):
        download_images(data_dir + '/train_test', model, limit)

## Move and rename images according to some silly scheme

In [ ]:
for model in cars_tba:
    move_images(data_dir, model)
    rename_images(data_dir + '/train', model, anno_train)
    rename_images(data_dir + '/test' , model, anno_test)

    names.append(model)

## Split images into train, test and validation

In [ ]:
redistribute_images(data_dir, cars_tba, dir2='/val/', train_test_ratio=0.9)